In [ ]:
import json
import base64
import re
import os

def create_json_from_markdown(markdown_path, output_filename="request.json"):
    """
    Parses a markdown file to create a JSON request for a journal entry.

    Args:
        markdown_path (str): The file path to the markdown journal entry.
        output_filename (str): The name of the output JSON file.
    """
    # --- 1. Read and Parse the Markdown File ---
    try:
        with open(markdown_path, 'r') as md_file:
            lines = md_file.readlines()
    except FileNotFoundError:
        print(f"Error: The file '{markdown_path}' was not found.")
        return

    title = ""
    paragraphs = []
    images_to_process = []
    current_paragraph = ""

    # Regular expression to find markdown image tags
    image_regex = re.compile(r'!\[.*\]\((.*?)\)')

    for line in lines:
        line = line.strip()
        
        # Find title
        if line.startswith('### '):
            title = line[4:].strip()
            continue

        # Find image
        image_match = image_regex.match(line)
        if image_match:
            # When an image is found, if there's a current paragraph, add it to the list
            if current_paragraph:
                paragraphs.append(current_paragraph)
                current_paragraph = ""
            
            image_path = image_match.group(1)
            # The position is after the last paragraph found (0-indexed)
            position = len(paragraphs) - 1
            images_to_process.append({"path": image_path, "position": position})
            continue

        # Collate paragraphs
        if line:
            current_paragraph += line + " "
        elif current_paragraph:
            # A blank line signifies the end of a paragraph
            paragraphs.append(current_paragraph.strip())
            current_paragraph = ""
    
    # Add any remaining paragraph
    if current_paragraph:
        paragraphs.append(current_paragraph.strip())

    # The full text is all paragraphs joined together
    full_text = "\n\n".join(paragraphs)

    # --- 2. Process and Encode Images ---
    processed_images = []
    for image_info in images_to_process:
        image_path = image_info["path"]
        try:
            image_format = image_path.split('.')[-1]
            with open(image_path, "rb") as image_file:
                base64_bytes = base64.b64encode(image_file.read())
                encoded_image_string = base64_bytes.decode('utf-8')
            
            processed_images.append({
                "encoding": "base64",
                "format": image_format,
                "content": encoded_image_string,
                "position_after_paragraph": image_info["position"]
            })
            print(f"Successfully encoded '{image_path}' to be placed after paragraph {image_info['position']}.")
        except FileNotFoundError:
            print(f"Warning: Image file '{image_path}' not found. Skipping.")
        except Exception as e:
            print(f"An error occurred encoding '{image_path}': {e}")
            
    # --- 3. Structure and Write the JSON File ---
    request_data = {
        "entry_data": {
            "title": title,
            "text": full_text,
        },
        "media_context": {
            "video_emotion": "happy", # Example value
            "video_emotion_confidence": 0.955, # Example value
            "images": processed_images
        }
    }

    try:
        with open(output_filename, 'w') as json_file:
            json.dump(request_data, json_file, indent=2)
        print(f"\nSuccessfully created '{output_filename}'")
    except Exception as e:
        print(f"An error occurred while writing the JSON file: {e}")

In [ ]:
markdown_file_to_parse = "journal.md"
create_json_from_markdown(markdown_file_to_parse)

In [ ]:
import json
import base64
import io
from PIL import Image
import matplotlib.pyplot as plt

def display_image_from_response(file_path: str):
    """
    Reads a JSON response file, decodes a base64 image,
    and displays it using matplotlib.
    """

    # 1. Open and read the JSON file
    with open(file_path, 'r') as f:
        response_data = json.load(f)

    # 2. Extract the base64 string from the 'images' list
    if not response_data.get('images'):
        print("Error: No 'images' key found in the JSON file.")
        return
        
    base64_image_str = response_data['images'][0]

    # 3. Decode the base64 string into bytes
    image_bytes = base64.b64decode(base64_image_str)

    # 4. Create an in-memory byte stream and open it as an image
    image_file = io.BytesIO(image_bytes)
    image = Image.open(image_file)

    # 5. Plot the image using matplotlib
    print(f"Displaying image from prompt: \"{response_data.get('prompt', 'N/A')}\"")
    
    plt.figure()
    plt.imshow(image)
    plt.axis('off')  # Hide the axes for a cleaner look
    plt.show()


json_file_path = 'response.json'
display_image_from_response(json_file_path)